(1) beomi/Llama-3-Open-Ko-8B, unsloth, ko alpaca data

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "beomi/Llama-3-Open-Ko-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
PROMPT_DICT = {
    "prompt_input": """다음은 작업을 설명하는 지침 사항과 추가적인 입력을 제공합니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
    ### 지침 사항:
    {instruction}

    ### 입력:
    {input}

    ### 응답:""",
    "prompt_no_input": """다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
    ### 지침 사항:
    {instruction}

    ### 응답:"""
}

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    texts = []
    for i in range(len(examples["instruction"])):
        if examples["input"][i] != "":
            source = PROMPT_DICT["prompt_input"].format(instruction=examples["instruction"][i], input=examples["input"][i])
        else:
            source = PROMPT_DICT["prompt_no_input"].format(instruction=examples["instruction"][i])

        target = f"{examples['output'][i]}{EOS_TOKEN}"
        text = f"{source} {target}"

        texts.append(text)

    return {"text": texts}
pass

from datasets import load_dataset
dataset = load_dataset("Bingsu/ko_alpaca_data", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# 출력 확인
print("출력 확인:::")
print(dataset[0])
print(dataset[1])

train_dataset = dataset.select(range(10000))
print(train_dataset[0])

출력 확인:::
{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.', 'text': '다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.\n    ### 지침 사항:\n    건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n    ### 응답: 세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<|end_of_text|>'}
{'instruction': '세 가지 기본 색은 무엇인가요?', 'input': '', 'output': '기본 색은 빨강, 파랑, 노랑입니다.', 'text': '다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.\n    ### 지침 사항:\n    세 가지 기본 색은 무엇인가요?\n\n    ### 응답: 기본 색은 빨강, 파랑, 노랑입니다.<|end_of_text|>'}
{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.', 'text': '다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.\n    ### 지침 사항:\n    건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n    ### 응답: 세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<|end_of_text|>'}


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=1,    # epoch 수
        per_device_train_batch_size=16,    # 배치 크기
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
8.582 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

Counting untrained tokens:   0%|          | 0/10000 [00:00<?, ? examples/s]

Unsloth: Setting embed_tokens & lm_head untrained tokens to mean(trained) to counteract NaNs during training.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 156
 "-____-"     Number of trainable parameters = 1,092,616,192


Step,Training Loss
1,2.635800
2,2.587600
3,2.450300
4,2.177000
5,1.859700
6,1.542100
7,1.264200
8,1.229600
9,1.196000
10,1.225400


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
    [PROMPT_DICT["prompt_no_input"].format(
        instruction="줄리어스 시저는 어떻게 죽었나요?"
    )],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

NameError: name 'FastLanguageModel' is not defined

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
if False: # change False->True, "username/modelname" to yours
    model.push_to_hub("username/finetuning-2",
                      use_auth_token=True,
                      commit_message="Initial commit",
                      private=False)

README.md:   0%|          | 0.00/565 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Saved model to https://huggingface.co/lsef/finetuning-2


## Load adapters from the Hub

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

T4에서 실행하려면: 제일 위 블럭 pip 실행 후 런타임->세션 다시 시작


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lsef/finetuning-2", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

## Inference

In [20]:
PROMPT_DICT = {
    "prompt_input": """다음은 작업을 설명하는 지침 사항과 추가적인 입력을 제공합니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
{instruction}

### 입력:
{input}

### 응답:""",
    "prompt_no_input": """다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
{instruction}

### 응답:"""
}

# input이 없는 inference
if True:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_no_input"].format(
          instruction="줄리어스 시저는 누구한테 배신당했나요?"
      )],
      return_tensors="pt"
  ).to("cuda")

# input이 있는 inference
if False:
  inputs = tokenizer(
      [PROMPT_DICT["prompt_input"].format(
          instruction="다음을 동물, 식물, 광물로 분류하십시오.",
          input="참나무, 구리, 코끼리"
      )],
      return_tensors="pt"
  ).to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 128,
    eos_token_id=tokenizer.eos_token_id, # EOS 토큰을 사용하여 명시적으로 출력의 끝을 지정.
    use_cache = True
)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
result = decoded_outputs[0]

print(result)  # print() 함수를 사용하지 않을 시 줄바꿈 문자 '\n' 반영 안됨

다음은 작업을 설명하는 지침 사항입니다. 요청을 적절하게 완료하는 응답을 한국어로 작성하세요.
### 지침 사항:
줄리어스 시저는 누구한테 배신당했나요?

### 응답: 줄리어스 시저는 브루투스에게 배신당했습니다.
